In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install pytorch-lightning
!pip install segmentation-models-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 705 kB 32.0 MB/s 
     |████████████████████████████████| 419 kB 66.0 MB/s 
     |████████████████████████████████| 5.9 MB 52.9 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 7.6 MB/s 
     |████████████████████████████████| 376 kB 57.3 MB/s 
     |████████████████████████████████| 

In [ ]:
import pytorch_lightning as pl
import pandas as pd
import cv2
import os 
import torchvision
from torch import nn
from torch.utils.data import Dataset ,DataLoader, random_split
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torchmetrics
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torchvision.utils import make_grid
import math
import torch.nn.functional as F
from torchvision.ops import box_convert
from PIL import Image
from pycocotools.coco import COCO
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import random
import matplotlib.image as mpimg
from imutils import paths
import segmentation_models_pytorch as smp
import sys

In [ ]:
sys.path.insert(0, "/content/drive/MyDrive/Github_Repos/historia_ml_handwritten_text_line_segmentation")
sys.path.insert(1, "/content/drive/MyDrive/Github_Repos/historia_ml_handwritten_text_line_segmentation/src/utils")

In [ ]:
!pip install rich
!pip install hydra-core

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 32.3 MB/s 
     |████████████████████████████████| 51 kB 8.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 151 kB 36.0 MB/s 
     |████████████████████████████████| 117 kB 70.8 MB/s 
     |████████████████████████████████| 79 kB 9.0 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=396704c3452a673a0634a5bfdd83319a967895e62b7af05cc1956d0e27c7aa21
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime


In [ ]:
import src
from src.custom_segmentation_dataset import SegmentationDataset
from src.config import cfg, transform
import src.utils as utils
from src.datamodules.segmentation_datamodule import LitDataModule
from src.models.unet_module import LitModel

In [ ]:
train_image_set = utils.get_file_paths.train_image_set
train_mask_set = utils.get_file_paths.train_mask_set
val_image_set = utils.get_file_paths.val_image_set
val_mask_set = utils.get_file_paths.val_mask_set
test_image_set = utils.get_file_paths.test_image_set
test_mask_set = utils.get_file_paths.test_mask_set

In [ ]:
print(f"len(train_image_set): {len(train_image_set)}\nlen(train_mask_set): {len(train_mask_set)}:")
print(f"len(val_image_set): {len(val_image_set)}\nlen(val_mask_set): {len(val_mask_set)}:")
print(f"len(test_image_set): {len(test_image_set)}\nlen(test_mask_set): {len(test_mask_set)}:")

len(train_image_set): 5000
len(train_mask_set): 5000:
len(val_image_set): 600
len(val_mask_set): 600:
len(test_image_set): 1500
len(test_mask_set): 1500:


In [ ]:
dm = LitDataModule()
dm.setup()

In [ ]:
model = LitModel()
trainer = pl.Trainer(max_epochs=cfg["num_epochs"],
                  accelerator="auto",
                  devices=1 if torch.cuda.is_available() else None,
                  callbacks=[LearningRateMonitor(logging_interval="step"),
                                TQDMProgressBar(refresh_rate=10),
                             EarlyStopping(monitor="val_loss", mode="min")],
)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, dm)

In [ ]:
trainer.validate(model, dm)

In [ ]:
trainer.test(model, dm)

In [ ]:
# serialize the model to disk
torch.save(model, cfg["base_output"] + "/pyimagesearch_example_lightning.pth")

In [ ]:
test_img_filenames = os.listdir(cfg["test_image_dataset"])
img_paths = [cfg["test_image_dataset"] + "/" + file for file in test_img_filenames]

In [ ]:
from src.utils.make_prediction import make_predictions

In [ ]:
# load the image paths in out testing file and randomly select 10 image paths
print("[INFO] loading up test image paths...")
img_paths = np.random.choice(img_paths, size=10)

# load our model from disk and flash it to the current device
print("[INFO] load up model...")
model = torch.load(cfg["base_output"] + "/pyimagesearch_example_lightning_testLoss0_25.pth").to(cfg["device"])

# iterate over the randomly selected test image paths
for path in img_paths:
  # make predictions and visualize the results
  make_predictions(model, path)

In [ ]:
high_resolution_test_img_files = os.listdir("/content/drive/MyDrive/Datasets/mask-image/original")
high_resolution_test_img_files_paths = ["/content/drive/MyDrive/Datasets/mask-image/original/" + file for file in high_resolution_test_img_files]

In [ ]:
# load the image paths in out testing file and randomly select 10 image paths
print("[INFO] loading up test image paths...")
high_resolution_test_img_files_paths = np.random.choice(high_resolution_test_img_files_paths, size=10)

# load our model from disk and flash it to the current device
print("[INFO] load up model...")
model = torch.load(cfg["base_output"] + "/pyimagesearch_example_lightning_testLoss0_25.pth").to(cfg["device"])
# iterate over the randomly selected test image paths
for path in high_resolution_test_img_files_paths:
  # make predictions and visualize the results
  make_predictions(model, path)

In [ ]:
prediction_patches = []
img_patches_folder_path = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page0/img_patches"
img_patches_filenames = os.listdir(img_patches_folder_path)
img_patches_path = [os.path.join(img_patches_folder_path, file) for file in img_patches_filenames]

In [ ]:
for path in img_patches_path:
  prediction_patches.append(make_predictions(model, path))

In [ ]:
x = prediction_patches[27]["predicted_patch"][208]
x

array([255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 25

In [ ]:
x = prediction_patches[17]["predicted_patch"]
x

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255]], dtype=uint8)

In [ ]:
def get_patch_num(patches, idx):
  x = patches[idx]["filename"]
  y = x.split("_")
  z = int(y[-1].split(".")[0])
  return z

def get_patch_num_from_name(name):
  x = name.split("_")
  y = int(x[-1].split(".")[0])
  return y

In [ ]:
patch_num = get_patch_num(prediction_patches, 2)
patch_num

IndexError: ignored

In [ ]:
patch_names = []
for d in prediction_patches:
    patch_names.append(d["filename"])
patch_names

[]

In [ ]:
patch_nums = []
for name in patch_names:
  patch_nums.append(get_patch_num_from_name(name))
patch_nums

[]

In [ ]:
patch_num_name_map = list(zip(patch_nums, patch_names))
type(patch_num_name_map)

list

In [ ]:
import operator

sorted_zip = sorted(patch_num_name_map, key=operator.itemgetter(0))
sorted_zip

[]

In [ ]:
patch_num_dict_map = list(zip(patch_nums, prediction_patches))
patch_num_dict_map

[]

In [ ]:
sorted_patch_num_dict_map = sorted(patch_num_dict_map, key=operator.itemgetter(0))
sorted_patch_num_dict_map

[]

In [ ]:
sorted_prediction_patches = []

In [ ]:
sorted_patch_num_dict_map[0][1]

IndexError: ignored

In [ ]:
for x in sorted_patch_num_dict_map:
  print(x[1]["filename"])

In [ ]:
sorted_patch_num_dict_map[49][1]["predicted_patch"]

In [ ]:
for zipp in sorted_patch_num_dict_map:
  sorted_prediction_patches.append(zipp[1]["predicted_patch"])
sorted_prediction_patches[1]

In [ ]:
def sort_predicted_patch_masks(pred_patches):
  patch_names = []
  for d in prediction_patches:
      patch_names.append(d["filename"])
  patch_names

  patch_nums = []
  for name in patch_names:
    patch_nums.append(get_patch_num_from_name(name))
  patch_nums

  patch_num_dict_map = list(zip(patch_nums, prediction_patches))
  sorted_patch_num_dict_map = sorted(patch_num_dict_map, key=operator.itemgetter(0))
  
  sorted_prediction_patches = []

  for zipp in sorted_patch_num_dict_map:
    sorted_prediction_patches.append(zipp[1]["predicted_patch"])

  return sorted_prediction_patches

In [ ]:
from src.utils.concatenate_patches_to_image import from_patches_to_image
from src.utils.resize_image_according_to_window_size import resize_to_multiple_of_window_scale
from src.utils.display_image_from_array import img_show_from_arr

In [ ]:
page0_img_path = "/content/drive/MyDrive/Datasets/mask-image/original/0.png"
page0_img = cv2.imread(page0_img_path)
page0_resized_img = resize_to_multiple_of_window_scale(page0_img, 320)
page0_resized_img.shape

In [ ]:
page0_predicted_mask = from_patches_to_image(sorted_prediction_patches, page0_resized_img, 320)
type(page0_predicted_mask)

In [ ]:
page0_predicted_mask.shape

In [ ]:
page0_expanded = np.expand_dims(page0_predicted_mask, axis=2)
page0_expanded.shape

In [ ]:
page0_expanded_RGB = cv2.cvtColor(page0_expanded, cv2.COLOR_GRAY2BGR)
page0_expanded_RGB.shape

In [ ]:
img_show_from_arr(page0_expanded_RGB)

In [ ]:
cv2.imwrite("/content/drive/MyDrive/output/test_page_predicted_masks/page0_predicted_mask.png", page0_expanded_RGB)

True

In [ ]:
def get_img_patch_paths(parent_folder_path):
  img_patches_filenames = os.listdir(parent_folder_path + "/img_patches")
  img_patches_folder_path = parent_folder_path + "/img_patches"
  img_patches_path = [os.path.join(img_patches_folder_path, file) for file in img_patches_filenames]

  return img_patches_path
          

In [ ]:
def get_patch_num(patches, idx):
  x = patches[idx]["filename"]
  y = x.split("_")
  z = int(y[-1].split(".")[0])
  return z

def get_patch_num_from_name(name):
  x = name.split("_")
  y = int(x[-1].split(".")[0])
  return y

In [ ]:
def sort_predicted_patch_masks(pred_patches):
  patch_names = []
  for d in pred_patches:
      patch_names.append(d["filename"])
  patch_names

  patch_nums = []
  for name in patch_names:
    patch_nums.append(get_patch_num_from_name(name))
  patch_nums

  patch_num_dict_map = list(zip(patch_nums, pred_patches))
  sorted_patch_num_dict_map = sorted(patch_num_dict_map, key=operator.itemgetter(0))
  
  sorted_prediction_patches = []

  for zipp in sorted_patch_num_dict_map:
    sorted_prediction_patches.append(zipp[1]["predicted_patch"])

  return sorted_prediction_patches

In [ ]:
def get_predicted_mask_from_patches(test_img_path, window_size, pred_patches):
  img = cv2.imread(test_img_path)
  img = resize_to_multiple_of_window_scale(img, window_size)
  sorted_pred_patches = sort_predicted_patch_masks(pred_patches)
  img_predicted_mask = from_patches_to_image(sorted_pred_patches, img, window_size)
  img_predicted_mask_expanded = np.expand_dims(img_predicted_mask, axis=2)
  img_predicted_mask_expanded_RGB = cv2.cvtColor(img_predicted_mask_expanded, cv2.COLOR_GRAY2RGB)
  return img_predicted_mask_expanded_RGB


In [ ]:
prediction_patches = []
parent_folder = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page0"
for path in img_patches_path:
  prediction_patches.append(make_predictions(model, path))

In [ ]:
page0 = "/content/drive/MyDrive/Datasets/mask-image/original/0.png"
test_image_pred_mask = get_predicted_mask_from_patches(page0, 320, prediction_patches)
img_show_from_arr(test_image_pred_mask)

In [ ]:
from src.utils.display_image_from_path import img_show

In [ ]:
img_show("/content/drive/MyDrive/Datasets/mask-image/labels/0.png")

In [ ]:
from src.utils.create_patches_with_path import create_patches

In [ ]:
page1_img_path = "/content/drive/MyDrive/Datasets/mask-image/original/1.png"
window_size = 224
# create image and mask patches to create patch folders for model
page1_img_patches = create_patches(page1_img_path, window_size)
# create img_patches and mask_patches folders under page1 folder
page1_folder_path = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page1"
if not (os.path.exists(page1_folder_path)):
        os.mkdir(page1_folder_path)
if not (os.path.exists(page1_folder_path + "/img_patches")):
        os.mkdir(page1_folder_path + "/img_patches")


In [ ]:
page1_img_patches[0].shape

(224, 224, 3)

In [ ]:
from src.utils.create_images import create_img_folder

In [ ]:
# create img and mask patch images in img_patches and mask_patches folders
page1_img_patch_folder_path = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page1/img_patches"
create_img_folder(page1_img_patch_folder_path, "page1", page1_img_patches)

In [ ]:
# Now feed the model with these img and mask patches and get prediction patches
page1_prediction_patches = []
page1_parent_folder = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page1"
page1_img_patches_path = get_img_patch_paths(page1_parent_folder)


In [ ]:
for path in page1_img_patches_path:
  page1_prediction_patches.append(make_predictions(model, path))

RuntimeError: ignored

In [ ]:
page1_pred_mask = get_predicted_mask_from_patches(page1_img_path, 224, page1_prediction_patches)
img_show_from_arr(page1_pred_mask);

In [ ]:
cv2.imwrite("/content/drive/MyDrive/output/test_page_predicted_masks/page1_predicted_mask_from_224x224_patches.png", page1_pred_mask)

True

In [ ]:
img_show("/content/drive/MyDrive/Datasets/mask-image/original/1.png")

In [ ]:
img_show("/content/drive/MyDrive/Datasets/mask-image/labels/1.png")

In [ ]:
page1_img_path = "/content/drive/MyDrive/Datasets/mask-image/original/1.png"
window_size = 320
# create image and mask patches to create patch folders for model
page1_320_img_patches = create_patches(page1_img_path, window_size)
# create img_patches and mask_patches folders under page1 folder
page1_320_folder_path = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page1_320"
if not (os.path.exists(page1_320_folder_path)):
        os.mkdir(page1_320_folder_path)
if not (os.path.exists(page1_320_folder_path + "/img_patches")):
        os.mkdir(page1_320_folder_path + "/img_patches")

In [ ]:
# create img and mask patch images in img_patches and mask_patches folders
page1_320_img_patch_folder_path = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page1_320/img_patches"
create_img_folder(page1_320_img_patch_folder_path, "page1", page1_320_img_patches)

In [ ]:
# Now feed the model with these img and mask patches and get prediction patches
page1_320_prediction_patches = []
page1_320_parent_folder = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page1_320"
page1_320_img_patches_path = get_img_patch_paths(page1_320_parent_folder)

In [ ]:
for path in page1_320_img_patches_path:
  page1_320_prediction_patches.append(make_predictions(model, path))

RuntimeError: ignored

In [ ]:
page1_320_pred_mask = get_predicted_mask_from_patches(page1_img_path, 320, page1_320_prediction_patches)
img_show_from_arr(page1_320_pred_mask)
cv2.imwrite("/content/drive/MyDrive/output/test_page_predicted_masks/page1_predicted_mask_from_320x320_patches.png", page1_320_pred_mask)

In [ ]:
  patch_names = []
  for d in page1_320_prediction_patches:
      patch_names.append(d["filename"])
  patch_names

  patch_nums = []
  for name in patch_names:
    patch_nums.append(get_patch_num_from_name(name))
  patch_nums

  patch_num_dict_map = list(zip(patch_nums, page1_320_prediction_patches))
  sorted_patch_num_dict_map = sorted(patch_num_dict_map, key=operator.itemgetter(0))
  
  sorted_prediction_patches = []

  for zipp in sorted_patch_num_dict_map:
    sorted_prediction_patches.append(zipp[1]["predicted_patch"])

In [ ]:
patch_names

In [ ]:
patch_nums

In [ ]:
sorted_patch_num_dict_map

In [ ]:
page1_img_path = "/content/drive/MyDrive/Datasets/mask-image/original/1.png"
window_size = 480
# create image patches to create patch folder for model
page1_480_img_patches = create_patches(page1_img_path, window_size)
# create img_patches folder under page1 folder
page1_480_folder_path = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page1_480"
if not (os.path.exists(page1_480_folder_path)):
        os.mkdir(page1_480_folder_path)
if not (os.path.exists(page1_480_folder_path + "/img_patches")):
        os.mkdir(page1_480_folder_path + "/img_patches")


In [ ]:
# create img and mask patch images in img_patches and mask_patches folders
page1_480_img_patch_folder_path = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page1_480/img_patches"
create_img_folder(page1_480_img_patch_folder_path, "page1", page1_480_img_patches)

In [ ]:
# Now feed the model with these img and mask patches and get prediction patches
page1_480_prediction_patches = []
page1_480_parent_folder = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page1_480"
page1_480_img_patches_path = get_img_patch_paths(page1_480_parent_folder)

In [ ]:
for path in page1_480_img_patches_path:
  page1_480_prediction_patches.append(make_predictions(model, path, page1_480_mask_path))

In [ ]:
page1_480_pred_mask = get_predicted_mask_from_patches(page1_img_path, 480, page1_480_prediction_patches)
img_show_from_arr(page1_480_pred_mask)
cv2.imwrite("/content/drive/MyDrive/output/test_page_predicted_masks/page1_predicted_mask_from_480x480_patches.png", page1_480_pred_mask)

In [ ]:
page2_img_path = "/content/drive/MyDrive/Datasets/mask-image/original/2.png"
window_size = 320
# create image and mask patches to create patch folders for model
page2_img_patches = create_patches(page2_img_path, window_size)
# create img_patches and mask_patches folders under page1 folder
page2_folder_path = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page2"
if not (os.path.exists(page2_folder_path)):
        os.mkdir(page2_folder_path)
if not (os.path.exists(page2_folder_path + "/img_patches")):
        os.mkdir(page2_folder_path + "/img_patches")


In [ ]:
# create img and mask patch images in img_patches and mask_patches folders
page2_img_patch_folder_path = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page2/img_patches"
create_img_folder(page2_img_patch_folder_path, "page2", page2_img_patches)

In [ ]:
# Now feed the model with these img and mask patches and get prediction patches
page2_prediction_patches = []
page2_parent_folder = "/content/drive/MyDrive/Datasets/berat_kurar_test_images/page2"
page2_img_patches_path = get_img_patch_paths(page2_parent_folder)

In [ ]:
for path in page2_img_patches_path:
  page2_prediction_patches.append(make_predictions(model, path, page2_mask_path))

In [ ]:
page2_pred_mask = get_predicted_mask_from_patches(page2_img_path, 320, page2_prediction_patches)
img_show_from_arr(page2_pred_mask)
cv2.imwrite("/content/drive/MyDrive/output/test_page_predicted_masks/page2_predicted_mask_from_320x320_patches.png", page2_pred_mask)